In [ ]:
import pandas as pd
from openai import OpenAI
import json
import tqdm
import matplotlib.pyplot as plt
import datetime
import concurrent.futures
from collections import Counter
import copy


client = OpenAI(
    # This is the default and can be omitted
    api_key="",
    base_url="https://api.chatanywhere.tech/v1"
)
secret_word_used = set(["Leonardo DiCaprio", "Taylor Swift", "Beyoncé"])
topic = "movie star"

In [3]:
system_prompt = f"""
Now we will play a game called Chameleon. I want you to be the game organizer.
Your task is to update and process game information.
Imagine we have 4 players. You should randomly assign a role to each player. Which means
there should be one Chameleon among all players.
Each player is controlled by real person. I will update their choices after each
round. You should not say any unrelated words. Now let's begin. You should randomly give a topic and secret word.
The format should be in a json foramt:
{{"Topic": {topic},  
"Secret Word": "",
"Roles":{{"player1": "non-chameleon", "player2":"non-chameleon", "player3":"chameleon", "player4":"non-chameleon"}}
}}
You should assign a secret word based on topic {topic}, don't use the word in {secret_word_used}. Also the roles should be randomized.
Don't let player1 be chameleon.
"""

In [ ]:
print(system_prompt)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        }
    ],
    model="gpt-4o-mini",
)
response = chat_completion.choices[0].message.content
print(response)

In [ ]:
cleaned_string = response.replace('\n', '')
system_game_info = json.loads(response)
system_game_info

In [7]:
# system_game_info['Secret Word'] = "Edward Norton"

In [8]:
agent_prompt_discussion_stage = """
Now we will play a game called Chameleon. I want you to be one of the players.
If you are a chameleon, try to guess the secret word.
If you are not a chameleon, talk about something related to the topic and secret word, but do not reveal the secret word directly.
No matter the role you are, you should only say one word.
I will provide you the game info in a json format.
For example:
{'Topic': 'Fruits', 'Secret Word': 'Mango', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player2', 'discussion': {'player1': ['Tropical'], 'player2': [], 'player3': [], 'player4': []}}
"""

agent_prompt_discussion_stage = """
Now we will play a game called Chameleon. I want you to be one of the players.
If you are a chameleon, try to guess the secret word in heart. But don't say it out loud. Because your goal is to hide your identity.
You can say something similar like what other people said.
If you are not a chameleon, talk about something related to the topic and secret word, but do not reveal the secret word directly.
Try to be vague, because if you are too obvious, the chameleon could easily have guessed it. But don't be too vague. your answer should be related to
topic and secret word. It's better to be specific but not revealing too much.
No matter each role, you should just say one short sentence. You shouldn't mention topic and secret word. 
And the sentence can be just few words. Don't be grammarly correct. Say it just like normal people do.
I will provide you the game info in a json format.
For example:
{'Topic': 'Fruits', 'Secret Word': 'Mango', 'Roles': 'non-chameleon', 'Round': 1, 'your position': 'player2', 'discussion': {'player1': ['Tropical'], 'player2': [], 'player3': [], 'player4': []}}
"""

In [9]:
discussion_history = {"player1": [], "player2":[], "player3":[], "player4":[]}
roles = system_game_info['Roles']
game_info_archive = {"round1": {"player1": [], "player2":[], "player3":[], "player4":[]}}
chameleon_player = [key for key, value in roles.items() if value == 'chameleon'][0]
secret_word_used.add(system_game_info['Secret Word'])

In [ ]:
for player in discussion_history.keys():
    role = roles[player]
    if role == 'non-chameleon':
        game_info = {'Topic': system_game_info['Topic'], 'Secret Word': system_game_info['Secret Word'], 'Roles': role, "Round": 1,
                            "your position": player,
                            "discussion":discussion_history}
    else:
        game_info = {'Topic': '', 'Secret Word': '', 'Roles': role, "Round": 1,
                            "your position": player,
                            "discussion":discussion_history}
    print(game_info)
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": agent_prompt_discussion_stage,
            },
            {
                "role":"user",
                "content": f"game info{game_info}"
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content
    print(response)
    discussion_history[player].append(response)
    game_info["discussion"] = copy.deepcopy(discussion_history)
    game_info_archive['round1'][player].append(game_info)

In [ ]:
discussion_history

In [ ]:
game_info_public = {"Round": 1, "discussion":discussion_history}
game_info_public

In [13]:
voting_prompt=f"""game info: {game_info_public}
Now the discussion stage on round 1 is over. Now it's the voting stage.
Each player should vote out the player whom you think is most like a chameleon.
give your reason, then
Vote a player by saying its number. You should state one number.
The format should be in a python array : [a single number you voted, your reasoning]
In your reasoning, state your role and player number first, then think. If you are chameleon, try to guess the secret word and topic.
"""

In [ ]:
for player in discussion_history.keys():
    role = roles[player]
    game_info = game_info_archive['round1'][player]
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": agent_prompt_discussion_stage,
            },
            {
                "role" : "user",
                "content": f"game info for round1 discussion stage: {game_info}"
            },
            {
                "role" : "user",
                "content": voting_prompt
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content
    print(response)
    

In [ ]:
role = roles[chameleon_player]
game_info = game_info_archive['round1'][chameleon_player]
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": agent_prompt_discussion_stage,
        },
        {
            "role" : "user",
            "content": f"game info for round1 discussion stage: {game_info}"
        },
        {
            "role" : "user",
            "content": f"""game info after round1: {game_info_public} This is the final round for chameleon player, which is you.
            in order for you to win,
            first state your role and which player you are, then do step by step reasoning to guess the topic and secret word
            at this stage, you can safely say it out loud"""
        }
    ],
    model="gpt-4o-mini",
)
response = chat_completion.choices[0].message.content
print(response)